In [1]:
import geopandas as gpd
import shapely
from getpass import getpass

import sys
import copy
sys.path.append("../Code")

from helper_functions import *
from loading_data import *
# from algorithms import *

WelkomCorne!


In [ ]:
dfob= get_dataframe("""
                    SELECT bk_votpand_cluster, COUNT(*)
                    FROM proj_afval_netwerk.rel_votpand_cluster_verblijfsobject
                    GROUP BY bk_votpand_cluster
                    """)

In [ ]:
dfob['split'] = dfob['bk_votpand_cluster'].str.split('~')
dfob['bag'] = dfob['split'].apply(lambda x: x[0]).astype('int64')
dfob['x'] = dfob['split'].apply(lambda x: x[1]).astype('float').round().astype('int')
dfob['y'] = dfob['split'].apply(lambda x: x[2]).astype('float').round().astype('int')
dfob = dfob.drop(['split'], axis=1)

In [ ]:
df_afstandn2 = get_dataframe("""
                                SELECT *
                                FROM proj_afval_netwerk.afv_rel_nodes_poi
                                """)

In [ ]:
df_afstandn2['split'] = df_afstandn2['bk_afv_rel_nodes_poi'].str.split('~')
df_afstandn2['x'] = df_afstandn2['split'].apply(lambda x: x[0]).astype('float').round().astype('int')
df_afstandn2['y'] = df_afstandn2['split'].apply(lambda x: x[1]).astype('float').round().astype('int')
df_afstandn2['type'] = df_afstandn2['split'].apply(lambda x: x[2])
verblijfsobjecten = df_afstandn2[df_afstandn2['type'] != 'afval_cluster']
verblijfsobjecten['bag'] = verblijfsobjecten['split'].apply(lambda x: x[3]).astype('int64')


In [ ]:
temp = dfob.set_index(['bag', 'x', 'y']).join(verblijfsobjecten.set_index(['bag', 'x', 'y']), how='outer').reset_index()

In [ ]:
df_afstandn = get_distance_matrix()

In [ ]:
joined= temp.set_index('s1_afv_nodes').join(df_afstandn.set_index('naar_s1_afv_nodes'), how='outer')
joined = joined.reset_index()[['van_s1_afv_nodes', 'index', 'afstand', 'count']].\
         rename(columns={'index':'naar_s1_afv_nodes'}).sort_values(by='afstand').\
         reset_index().drop(['index'],axis=1).dropna()

In [ ]:
joined

In [ ]:
def distance_matrix_with_counts():
    """
    Function that tries to match table with addresses per poi with 
    information and subsequently with distance matrix to give back
    a distance matrix with the amount of households per addres poi.
    """
    
    dfob= get_dataframe("""
                    SELECT bk_votpand_cluster, COUNT(*)
                    FROM proj_afval_netwerk.rel_votpand_cluster_verblijfsobject
                    GROUP BY bk_votpand_cluster
                    """)
    
    dfob['split'] = dfob['bk_votpand_cluster'].str.split('~')
    dfob['bag'] = dfob['split'].apply(lambda x: x[0]).astype('int64')
    dfob['x'] = dfob['split'].apply(lambda x: x[1]).astype('float').round().astype('int')
    dfob['y'] = dfob['split'].apply(lambda x: x[2]).astype('float').round().astype('int')
    dfob = dfob.drop(['split'], axis=1)
    
    df_afstandn2 = get_dataframe("""
                                SELECT *
                                FROM proj_afval_netwerk.afv_rel_nodes_poi
                                """)
    
    df_afstandn2['split'] = df_afstandn2['bk_afv_rel_nodes_poi'].str.split('~')
    df_afstandn2['x'] = df_afstandn2['split'].apply(lambda x: x[0]).astype('float').round().astype('int')
    df_afstandn2['y'] = df_afstandn2['split'].apply(lambda x: x[1]).astype('float').round().astype('int')
    df_afstandn2['type'] = df_afstandn2['split'].apply(lambda x: x[2])
    verblijfsobjecten = df_afstandn2[df_afstandn2['type'] != 'afval_cluster']
    verblijfsobjecten['bag'] = verblijfsobjecten['split'].apply(lambda x: x[3]).astype('int64')
    
    temp = dfob.set_index(['bag', 'x', 'y']).join(verblijfsobjecten.set_index(['bag', 'x', 'y']), how='outer').reset_index()
    df_afstandn = get_distance_matrix()
    joined= temp.set_index('s1_afv_nodes').join(df_afstandn.set_index('naar_s1_afv_nodes'), how='outer')
    joined = joined.reset_index()[['van_s1_afv_nodes', 'index', 'afstand', 'count']].\
             rename(columns={'index':'naar_s1_afv_nodes'}).sort_values(by='afstand').\
             reset_index().drop(['index'],axis=1).dropna()
    
    return joined

In [ ]:
joined = distance_matrix_with_counts()

### Try out

In [ ]:
all_households, rel_poi_df, joined, df_afstandn2 = initial_loading(use_count=True)

In [ ]:
df_afstandn2

In [ ]:
joined_cluster_distance = joined.set_index('s1_afv_nodes').join(df_afstandn2.set_index('van_s1_afv_nodes')).reset_index().rename(columns={'index': 'van_s1_afv_nodes'})

In [ ]:
good_result_rich = add_shortest_distances_to_all_households(all_households, joined_cluster_distance, count=True)
good_result_rich = good_result_rich[good_result_rich['uses_container']]
good_result_rich

In [ ]:
aansluitingen = create_aansluitingen(good_result_rich, joined_cluster_distance, use_count=True)

In [ ]:

def calculate_penalties(good_result, aansluitingen, use_count=False):
    """
    This function calculates all the penalties associated with the candidate
    solution. It does this by calculating the number of times all constraints
    are violated and applies the weighing that is associated with all these
    violations

    Input:
    dataframe good_result containing per adress or adress poi the distance
    to the nearest container for all fractions.
    dataframe aansluitingen containing for all clusters the amount of containers
    per fraction, the amount of people using these containers and the percentage
    of occupancy compared to the norm

    Output:
    The sum of all different penalties as a single float
    """
    
    penalty1 = good_result[good_result['rest_afstand'] > 100]
    penalty2 = good_result[good_result['plastic_afstand'] > 150]
    penalty3 = good_result[good_result['papier_afstand'] > 150]
    penalty4 = good_result[good_result['glas_afstand'] > 150]
    penalty5 = good_result[good_result['textiel_afstand'] > 300]
    penalty6 = aansluitingen[aansluitingen['rest_perc'] > 100]
    penalty7 = aansluitingen[aansluitingen['plastic_perc'] > 100]
    penalty8 = aansluitingen[aansluitingen['papier_perc'] > 100]
    penalty9 = aansluitingen[aansluitingen['glas_perc'] > 100]
    penalty10 = aansluitingen[aansluitingen['textiel_perc'] > 100]
    
    if not use_count:
        penalty1_sum = (penalty1['rest_afstand'].sum() - 100 * penalty1.shape[0])/good_result.shape[0] * 0.35
        penalty2_sum = (penalty2['plastic_afstand'].sum() - 150 * penalty2.shape[0])/good_result.shape[0] * 0.25
        penalty3_sum = (penalty3['papier_afstand'].sum() - 150 * penalty3.shape[0])/good_result.shape[0] * 0.2
        penalty4_sum = (penalty4['glas_afstand'].sum() - 150 * penalty4.shape[0])/good_result.shape[0] * 0.15
        penalty5_sum = (penalty5['textiel_afstand'].sum() - 300 * penalty5.shape[0])/good_result.shape[0] * 0.05
        penalty6_sum = (penalty6['poi_rest'] - (penalty6['rest'] * 100)).sum()/ good_result.shape[0] * 0.35 * 1000
        penalty7_sum = (penalty7['poi_plastic'] - (penalty7['plastic'] * 200)).sum()/ good_result.shape[0] * 0.25 * 1000
        penalty8_sum = (penalty8['poi_papier'] - (penalty8['papier'] * 200)).sum()/ good_result.shape[0] * 0.2 * 1000
        penalty9_sum = (penalty9['poi_glas'] - (penalty9['glas'] * 200)).sum()/ good_result.shape[0] * 0.15 * 1000
        penalty10_sum = (penalty10['poi_textiel'] - (penalty10['textiel'] * 750)).sum()/ good_result.shape[0] * 0.05 * 1000
        
    else:
        penalty1_sum = (penalty1['rest_afstand'].sum() - 100 * penalty1['count'].sum())/good_result['count'].sum() * 0.35
        penalty2_sum = (penalty2['plastic_afstand'].sum() - 150 * penalty2['count'].sum())/good_result['count'].sum() * 0.25
        penalty3_sum = (penalty3['papier_afstand'].sum() - 150 * penalty3['count'].sum())/good_result['count'].sum() * 0.2
        penalty4_sum = (penalty4['glas_afstand'].sum() - 150 * penalty4['count'].sum())/good_result['count'].sum() * 0.15
        penalty5_sum = (penalty5['textiel_afstand'].sum() - 300 * penalty5['count'].sum())/good_result['count'].sum() * 0.05
        penalty6_sum = (penalty6['poi_rest'] - (penalty6['rest'] * 100)).sum()/ good_result['count'].sum() * 0.35 * 1000
        penalty7_sum = (penalty7['poi_plastic'] - (penalty7['plastic'] * 200)).sum()/ good_result['count'].sum() * 0.25 * 1000
        penalty8_sum = (penalty8['poi_papier'] - (penalty8['papier'] * 200)).sum()/ good_result['count'].sum() * 0.2 * 1000
        penalty9_sum = (penalty9['poi_glas'] - (penalty9['glas'] * 200)).sum()/ good_result['count'].sum() * 0.15 * 1000
        penalty10_sum = (penalty10['poi_textiel'] - (penalty10['textiel'] * 750)).sum()/ good_result['count'].sum() * 0.05 * 1000
        
    total_penalties = sum([penalty1_sum, penalty2_sum, penalty3_sum, penalty4_sum, penalty5_sum,\
                           penalty6_sum, penalty7_sum, penalty8_sum, penalty9_sum, penalty10_sum])
    return total_penalties


In [ ]:
calculate_penalties(good_result_rich, aansluitingen, use_count=True)

### Initial test for correctly loading data using count per poi

In [2]:
all_households, rel_poi_df, joined, df_afstandn2 = initial_loading(use_count=True)

API data loaded
a
DB relation POIs loaded
b
Table all households created
API and DB joined
containers per cluster determined


../Code\loading_data.py:231: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  verblijfsobjecten['bag'] = verblijfsobjecten['split'].apply(lambda x: x[3]).astype('int64')


distance matrix loaded


In [4]:
joined, joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = \
best_of_random(50, joined, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True)
hill_dict, best_solution = hillclimber(1000, joined, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True, parameter='penalties')

Average distance is : 177.29976726577087
Penalites are: 288.2499546710465
Average distance is : 177.84272863295124
Penalites are: 293.0649943473652
Average distance is : 178.48921167869267
Penalites are: 291.74643907709043
Average distance is : 178.1243698954531
Penalites are: 287.77676966019015
Average distance is : 180.26088843943268
Penalites are: 295.12363870975713
Average distance is : 181.79668032727832
Penalites are: 297.00572417899053
Average distance is : 178.64827314387628
Penalites are: 291.2219657555895
Average distance is : 180.26332079923517
Penalites are: 291.25061653498096
Average distance is : 178.9856994009263
Penalites are: 294.77532013300174
Average distance is : 180.32581783468663
Penalites are: 295.733855449905
Average distance is : 178.80208943309162
Penalites are: 295.5639363623373
Average distance is : 177.82671833938568
Penalites are: 297.57473047507284
Average distance is : 178.4021414985456
Penalites are: 293.4712934428121
Average distance is : 181.744541679

Average distance is : 178.17013951444733
Penalites are: 288.0008290912037
288.0008290912037 287.4380122786924
Average distance is : 178.11433674781014
Penalites are: 287.93775173173367
287.93775173173367 287.4380122786924
Average distance is : 178.14825340682899
Penalites are: 287.71533186802026
287.71533186802026 287.4380122786924
Average distance is : 178.13261410972723
Penalites are: 287.7513818287438
287.7513818287438 287.4380122786924
Average distance is : 178.13632189728617
Penalites are: 287.6995031119021
287.6995031119021 287.4380122786924
Average distance is : 178.09001829937444
Penalites are: 287.6062863169691
287.6062863169691 287.4380122786924
Average distance is : 178.18658163028698
Penalites are: 287.73166751811385
287.73166751811385 287.4380122786924
Average distance is : 178.13449603767603
Penalites are: 287.77826081887247
287.77826081887247 287.4380122786924
Average distance is : 178.136909919623
Penalites are: 287.85974816801524
287.85974816801524 287.4380122786924
Av

Average distance is : 178.09906183305753
Penalites are: 287.7721063100476
287.7721063100476 287.148620845075
Average distance is : 178.1453668939114
Penalites are: 287.54573658009366
287.54573658009366 287.148620845075
Average distance is : 178.15384371178314
Penalites are: 287.86006636206827
287.86006636206827 287.148620845075
Average distance is : 178.15773702618577
Penalites are: 287.9223319971105
287.9223319971105 287.148620845075
Average distance is : 178.1225671679291
Penalites are: 287.73886652741385
287.73886652741385 287.148620845075
Average distance is : 178.18029419755095
Penalites are: 288.02251433864524
288.02251433864524 287.148620845075
Average distance is : 178.04453679209433
Penalites are: 287.8210687529869
287.8210687529869 287.148620845075
Average distance is : 178.1308029189306
Penalites are: 287.7979302403828
287.7979302403828 287.148620845075
Average distance is : 178.1424208440752
Penalites are: 287.8280711754915
287.8280711754915 287.148620845075
Average distanc

Average distance is : 178.12268860518387
Penalites are: 287.7405498288865
287.7405498288865 287.148620845075
Average distance is : 178.21888362296434
Penalites are: 287.7196057291409
287.7196057291409 287.148620845075
Average distance is : 178.12163202346804
Penalites are: 287.63047243645735
287.63047243645735 287.148620845075
Average distance is : 178.14893621122354
Penalites are: 287.88877173799597
287.88877173799597 287.148620845075
Average distance is : 178.15275310221014
Penalites are: 287.7401953302242
287.7401953302242 287.148620845075
Average distance is : 178.13312686160754
Penalites are: 287.91135885065614
287.91135885065614 287.148620845075
Average distance is : 178.1310943362181
Penalites are: 288.32727080003076
288.32727080003076 287.148620845075
Average distance is : 178.1243551828585
Penalites are: 287.79811177990916
287.79811177990916 287.148620845075
Average distance is : 178.1573257638519
Penalites are: 288.09786570919266
288.09786570919266 287.148620845075
Average di

Average distance is : 177.93866798957072
Penalites are: 287.5752897502239
287.5752897502239 287.148620845075
Average distance is : 178.14142709322672
Penalites are: 287.67704715781434
287.67704715781434 287.148620845075
Average distance is : 178.1203303265517
Penalites are: 287.8780760658196
287.8780760658196 287.148620845075
Average distance is : 178.12372706968387
Penalites are: 287.87100216432384
287.87100216432384 287.148620845075
Average distance is : 178.05405236947905
Penalites are: 287.7000547096016
287.7000547096016 287.148620845075
Average distance is : 178.18272086529504
Penalites are: 287.77528571408953
287.77528571408953 287.148620845075
Average distance is : 178.13902800529826
Penalites are: 287.676913738787
287.676913738787 287.148620845075
Average distance is : 178.134153714694
Penalites are: 287.82493999631953
287.82493999631953 287.148620845075
Average distance is : 178.09706063558951
Penalites are: 287.6948963500381
287.6948963500381 287.148620845075
Average distance

...

Average distance is : 178.13668752763823
Penalites are: 287.78784764176595
287.78784764176595 287.148620845075
Average distance is : 178.12301369381052
Penalites are: 287.9869500928352
287.9869500928352 287.148620845075
Average distance is : 178.00827924562543
Penalites are: 287.50670730679013
287.50670730679013 287.148620845075
Average distance is : 178.13003324557283
Penalites are: 287.81390657976414
287.81390657976414 287.148620845075
Average distance is : 178.0744725771507
Penalites are: 287.77494731052553
287.77494731052553 287.148620845075
Average distance is : 178.1321716664813
Penalites are: 287.7005274289591
287.7005274289591 287.148620845075
Average distance is : 178.12551802207858
Penalites are: 287.8427652285449
287.8427652285449 287.148620845075
Average distance is : 178.1453961600556
Penalites are: 287.8624718871365
287.8624718871365 287.148620845075
Average distance is : 178.01154701661042
Penalites are: 287.71625186219745
287.71625186219745 287.148620845075
Average dist

Average distance is : 178.14505711215654
Penalites are: 288.21265627489106
288.21265627489106 287.148620845075
Average distance is : 178.14060958268698
Penalites are: 287.8789601646314
287.8789601646314 287.148620845075
Average distance is : 178.21410128926124
Penalites are: 288.0154752448597
288.0154752448597 287.148620845075
Average distance is : 178.10378473749694
Penalites are: 287.74696751866315
287.74696751866315 287.148620845075
Average distance is : 178.14522534707496
Penalites are: 288.2297799789501
288.2297799789501 287.148620845075
Average distance is : 178.12077972314722
Penalites are: 287.4957382912279
287.4957382912279 287.148620845075
Average distance is : 178.18979618150072
Penalites are: 288.0444665463654
288.0444665463654 287.148620845075
Average distance is : 178.13097427766735
Penalites are: 287.85837162373724
287.85837162373724 287.148620845075
Average distance is : 178.10905230553072
Penalites are: 287.7463524950149
287.7463524950149 287.148620845075
Average dista

Average distance is : 178.10811525696272
Penalites are: 287.9013159899035
287.9013159899035 287.148620845075
Average distance is : 178.14373969799524
Penalites are: 287.64486559219523
287.64486559219523 287.148620845075
Average distance is : 178.10589407477
Penalites are: 287.7374801832534
287.7374801832534 287.148620845075
Average distance is : 178.09873609792413
Penalites are: 287.70139627725314
287.70139627725314 287.148620845075
Average distance is : 178.1469011122706
Penalites are: 287.65866901363967
287.65866901363967 287.148620845075
Average distance is : 178.17054672563478
Penalites are: 287.85472822217827
287.85472822217827 287.148620845075
Average distance is : 178.09819396819105
Penalites are: 287.69016324027837
287.69016324027837 287.148620845075
Average distance is : 178.14523727530613
Penalites are: 287.7654717731525
287.7654717731525 287.148620845075
Average distance is : 178.22260046112007
Penalites are: 287.88151599072745
287.88151599072745 287.148620845075
Average dis

Average distance is : 178.08735921749445
Penalites are: 287.6437682882615
287.6437682882615 287.148620845075
Average distance is : 178.12266766812283
Penalites are: 287.71761029262314
287.71761029262314 287.148620845075
Average distance is : 178.12163741451502
Penalites are: 287.77727654729927
287.77727654729927 287.148620845075
Average distance is : 178.2706891162824
Penalites are: 287.91190203413476
287.91190203413476 287.148620845075
Average distance is : 178.145556326975
Penalites are: 287.78232496717806
287.78232496717806 287.148620845075
Average distance is : 178.2128646113241
Penalites are: 287.7602537614739
287.7602537614739 287.148620845075
Average distance is : 178.08355655112152
Penalites are: 287.75375418857084
287.75375418857084 287.148620845075
Average distance is : 178.41695810770523
Penalites are: 287.6599968276726
287.6599968276726 287.148620845075
Average distance is : 178.24952842667778
Penalites are: 287.98524359903973
287.98524359903973 287.148620845075
Average dis

../Code\helper_functions.py:461: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = str(pd.datetime.now().date()) + '-' + str(pd.datetime.now().hour)


In [ ]:
joined, joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = \
best_of_random(10, joined, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True)
hill_dict, best_solution = hillclimber(500, joined, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True, parameter='penalties')
hill_df = pd.DataFrame.from_dict(hill_dict, orient='index')
hill_df.to_csv('hillclimber.csv')

In [38]:
def random_start_hillclimber(joined, all_households, rel_poi_df, df_afstandn2,\
                        clean=True, use_count=False, parameter = 'penalties'):
    i = int(input("How many random iterations?"))
    j = int(input("How many iterations hillclimber"))
    to_save = bool(input("Do you want the results saved(True/False)?"))
    
    joined, joined_cluster_distance, good_result_rich, aansluitingen, \
        avg_distance, penalties = best_of_random(i, joined,all_households, \
        rel_poi_df, df_afstandn2, clean=clean, use_count=clean)
    
    hill_df, best_solution = hillclimber(j, joined, all_households, \
        rel_poi_df, df_afstandn2, clean=clean, use_count=use_count,\
        parameter=parameter, save=to_save)
    return hill_df, best_solution

In [39]:
random_start_hillclimber(joined, all_households, rel_poi_df, df_afstandn2, use_count=True)

How many random iterations?2
How many iterations hillclimber5
Do you want the results saved(True/False)?False
Average distance is : 178.1243698954531
Penalites are: 287.77676966019015
Average distance is : 178.8238389173344
Penalites are: 290.3767736376332
Average distance is : 177.03092609582868
Penalites are: 288.9149878799533
***************************************
178.1243698954531 287.77676966019015
Average distance is : 178.1243698954531
Penalites are: 287.77676966019015
Average distance is : 178.13881135131658
Penalites are: 287.6520340537675
287.6520340537675 287.77676966019015
Average distance is : 178.08689227215302
Penalites are: 287.76560263646496
287.76560263646496 287.6520340537675
Average distance is : 178.1617771330432
Penalites are: 287.8865469353394
287.8865469353394 287.6520340537675
Average distance is : 178.13511424171156
Penalites are: 287.85743211219324
287.85743211219324 287.6520340537675
Average distance is : 178.20883651090634
Penalites are: 287.5346566649089


<ipython-input-35-5680e4cbb3b6>:57: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = str(pd.datetime.now().date()) + '-' + str(pd.datetime.now().hour)


(   avg_distance   penalties        best  amount of modifications
 0    178.124370  287.776770  287.776770                      NaN
 1    178.138811  287.652034  287.776770                      3.0
 2    178.086892  287.765603  287.652034                      5.0
 3    178.161777  287.886547  287.652034                      2.0
 4    178.135114  287.857432  287.652034                      1.0
 5    178.208837  287.534657  287.652034                      5.0,
       s1_afv_nodes  cluster_x  cluster_y  s1_afv_rel_nodes_poi  \
 0           483494     121815     490815                 288.0   
 1           483617     119304     482374                 412.0   
 2           483837     124925     481259                 633.0   
 3           483949     115010     486153                 744.0   
 4           483955     119333     485987                 750.0   
 ...            ...        ...        ...                   ...   
 5559        966892     124374     486050              483418.0   
 

In [8]:
hill_dict2, best_solution = hillclimber(5, best_solution, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True, parameter='penalties')

Average distance is : 178.1243698954531
Penalites are: 287.77676966019015
Average distance is : 178.19998420222174
Penalites are: 287.9701048131485
287.9701048131485 287.77676966019015
Average distance is : 178.27358899587205
Penalites are: 287.73719379212605
287.73719379212605 287.77676966019015
Average distance is : 178.3453783300627
Penalites are: 287.881715602907
287.881715602907 287.73719379212605
Average distance is : 178.08636298260183
Penalites are: 287.5638331822701
287.5638331822701 287.73719379212605
Average distance is : 178.03708711327033
Penalites are: 287.4837424660657
287.4837424660657 287.5638331822701


../Code\helper_functions.py:461: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = str(pd.datetime.now().date()) + '-' + str(pd.datetime.now().hour)


In [9]:
hill_dict3, best_solution = hillclimber(1000, best_solution, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True, parameter='penalties')

Average distance is : 178.03708711327033
Penalites are: 287.4837424660657
Average distance is : 178.0536327425079
Penalites are: 287.271327491307
287.271327491307 287.4837424660657
Average distance is : 178.12260517926254
Penalites are: 287.2385641263328
287.2385641263328 287.271327491307
Average distance is : 178.1347263106943
Penalites are: 287.0953919911779
287.0953919911779 287.2385641263328
Average distance is : 178.15285066646803
Penalites are: 287.0176884692078
287.0176884692078 287.0953919911779
Average distance is : 178.153537300802
Penalites are: 286.9254743618266
286.9254743618266 287.0176884692078
Average distance is : 178.05968516776483
Penalites are: 286.9247563158155
286.9247563158155 286.9254743618266
Average distance is : 178.07813333041457
Penalites are: 287.031197348978
287.031197348978 286.9247563158155
Average distance is : 177.9874012883654
Penalites are: 287.2651460767511
287.2651460767511 286.9247563158155
Average distance is : 178.05269196756925
Penalites are: 

Average distance is : 178.05114477587307
Penalites are: 287.5254913207319
287.5254913207319 286.9247563158155
Average distance is : 178.02014764694485
Penalites are: 287.51369312744794
287.51369312744794 286.9247563158155
Average distance is : 178.15003458525214
Penalites are: 287.6476443601237
287.6476443601237 286.9247563158155
Average distance is : 178.05567761139304
Penalites are: 287.5983484492675
287.5983484492675 286.9247563158155
Average distance is : 178.06303596574932
Penalites are: 287.1632949921172
287.1632949921172 286.9247563158155
Average distance is : 178.03999678820136
Penalites are: 287.4681516715831
287.4681516715831 286.9247563158155
Average distance is : 178.0335222382967
Penalites are: 287.4654889380531
287.4654889380531 286.9247563158155
Average distance is : 178.03990904603236
Penalites are: 287.44498174057776
287.44498174057776 286.9247563158155
Average distance is : 178.03908252781795
Penalites are: 287.48975760259935
287.48975760259935 286.9247563158155
Avera

Average distance is : 178.04161415288925
Penalites are: 287.49886978213937
287.49886978213937 286.9247563158155
Average distance is : 178.13223737980587
Penalites are: 287.5016227443516
287.5016227443516 286.9247563158155
Average distance is : 177.98967776715173
Penalites are: 287.6182361282774
287.6182361282774 286.9247563158155
Average distance is : 178.03708711327033
Penalites are: 287.4837424660657
287.4837424660657 286.9247563158155
Average distance is : 178.09524023166622
Penalites are: 287.5066447185555
287.5066447185555 286.9247563158155
Average distance is : 177.98178087523462
Penalites are: 287.6466057950594
287.6466057950594 286.9247563158155
Average distance is : 177.99993399364845
Penalites are: 287.40482086237864
287.40482086237864 286.9247563158155
Average distance is : 178.03581297877815
Penalites are: 287.48725823981147
287.48725823981147 286.9247563158155
Average distance is : 178.02893080361463
Penalites are: 287.10217795658167
287.10217795658167 286.9247563158155
Av

Average distance is : 178.06195010010453
Penalites are: 287.3753480977022
287.3753480977022 286.9247563158155
Average distance is : 178.01304125809304
Penalites are: 287.4283594933614
287.4283594933614 286.9247563158155
Average distance is : 178.04520651673178
Penalites are: 287.2520530039606
287.2520530039606 286.9247563158155
Average distance is : 178.0427230216904
Penalites are: 287.5747546925726
287.5747546925726 286.9247563158155
Average distance is : 178.07034185859263
Penalites are: 287.5100636470504
287.5100636470504 286.9247563158155
Average distance is : 178.0917718987419
Penalites are: 287.54895286278514
287.54895286278514 286.9247563158155
Average distance is : 178.05417055271582
Penalites are: 287.6092333542075
287.6092333542075 286.9247563158155
Average distance is : 178.05756292471995
Penalites are: 287.4321806166675
287.4321806166675 286.9247563158155
Average distance is : 178.06213754290548
Penalites are: 287.5848128574018
287.5848128574018 286.9247563158155
Average di

Average distance is : 178.03708711327033
Penalites are: 287.4837424660657
287.4837424660657 286.9247563158155
Average distance is : 178.09352109027003
Penalites are: 287.8255895022055
287.8255895022055 286.9247563158155
Average distance is : 178.02607714253023
Penalites are: 287.4389020863203
287.4389020863203 286.9247563158155
Average distance is : 178.0443552950073
Penalites are: 287.65383946100644
287.65383946100644 286.9247563158155
Average distance is : 177.97044791541276
Penalites are: 287.5203501384288
287.5203501384288 286.9247563158155
Average distance is : 177.99106797307348
Penalites are: 287.8664932694473
287.8664932694473 286.9247563158155
Average distance is : 177.99685061239228
Penalites are: 287.48162976467097
287.48162976467097 286.9247563158155
Average distance is : 178.04211285611385
Penalites are: 287.56900687886923
287.56900687886923 286.9247563158155
Average distance is : 178.07423294550225
Penalites are: 287.71702009827345
287.71702009827345 286.9247563158155
Ave

...

Average distance is : 178.00385562495444
Penalites are: 287.2992318023763
287.2992318023763 286.9247563158155
Average distance is : 178.03691132832577
Penalites are: 287.57241959729294
287.57241959729294 286.9247563158155
Average distance is : 177.95814814328233
Penalites are: 287.4336438629884
287.4336438629884 286.9247563158155
Average distance is : 178.07858258508068
Penalites are: 287.6029795638942
287.6029795638942 286.9247563158155
Average distance is : 178.03034280010996
Penalites are: 287.41444186401014
287.41444186401014 286.9247563158155
Average distance is : 178.08530256093385
Penalites are: 287.3408651527386
287.3408651527386 286.9247563158155
Average distance is : 178.0672213054537
Penalites are: 287.5338247295968
287.5338247295968 286.9247563158155
Average distance is : 178.04971835678518
Penalites are: 287.3517522796812
287.3517522796812 286.9247563158155
Average distance is : 178.02670721518106
Penalites are: 287.5318090670885
287.5318090670885 286.9247563158155
Average

Average distance is : 178.03292442305127
Penalites are: 287.4892459885851
287.4892459885851 286.9247563158155
Average distance is : 178.04100823820892
Penalites are: 287.5321143452371
287.5321143452371 286.9247563158155
Average distance is : 178.06156390651373
Penalites are: 287.6726045082755
287.6726045082755 286.9247563158155
Average distance is : 177.98331710823507
Penalites are: 287.2900922488094
287.2900922488094 286.9247563158155
Average distance is : 178.0832285846921
Penalites are: 287.75505260570304
287.75505260570304 286.9247563158155
Average distance is : 178.04070296242324
Penalites are: 287.47775369280885
287.47775369280885 286.9247563158155
Average distance is : 178.0351353727917
Penalites are: 287.6940351002236
287.6940351002236 286.9247563158155
Average distance is : 178.07193846294635
Penalites are: 287.65432522810545
287.65432522810545 286.9247563158155
Average distance is : 178.0803155112394
Penalites are: 287.46360737161
287.46360737161 286.9247563158155
Average dis

Average distance is : 178.06053988481207
Penalites are: 287.5695403901825
287.5695403901825 286.9247563158155
Average distance is : 177.98896939411924
Penalites are: 287.5044913811572
287.5044913811572 286.9247563158155
Average distance is : 177.98841044582684
Penalites are: 287.48916829265613
287.48916829265613 286.9247563158155
Average distance is : 178.0259263778011
Penalites are: 287.4889490436325
287.4889490436325 286.9247563158155
Average distance is : 177.74137551096462
Penalites are: 287.456931160495
287.456931160495 286.9247563158155
Average distance is : 178.02417097577896
Penalites are: 286.99823126129536
286.99823126129536 286.9247563158155
Average distance is : 178.04076800815938
Penalites are: 287.54059262646734
287.54059262646734 286.9247563158155
Average distance is : 178.07352624170954
Penalites are: 287.51965036419966
287.51965036419966 286.9247563158155
Average distance is : 178.02759279203443
Penalites are: 287.4815948910422
287.4815948910422 286.9247563158155
Avera

Average distance is : 178.07592637982214
Penalites are: 287.6892670989821
287.6892670989821 286.9247563158155
Average distance is : 178.0151101254099
Penalites are: 287.50366088231897
287.50366088231897 286.9247563158155
Average distance is : 178.04716222821355
Penalites are: 287.4127379424092
287.4127379424092 286.9247563158155
Average distance is : 178.04474463958385
Penalites are: 287.5955977873359
287.5955977873359 286.9247563158155
Average distance is : 177.96490689833857
Penalites are: 287.44479328784956
287.44479328784956 286.9247563158155
Average distance is : 178.01982474054518
Penalites are: 287.5515302373615
287.5515302373615 286.9247563158155
Average distance is : 178.02119685784197
Penalites are: 287.7313756145526
287.7313756145526 286.9247563158155
Average distance is : 178.07651532010152
Penalites are: 287.61446668644226
287.61446668644226 286.9247563158155
Average distance is : 178.04918350737398
Penalites are: 287.6327755777481
287.6327755777481 286.9247563158155
Avera

Average distance is : 178.0546887820119
Penalites are: 287.57697015167247
287.57697015167247 286.9247563158155
Average distance is : 178.00005072973255
Penalites are: 287.3846650708898
287.3846650708898 286.9247563158155
Average distance is : 178.15322966838164
Penalites are: 287.65448587421656
287.65448587421656 286.9247563158155
Average distance is : 178.03740503086163
Penalites are: 287.45682065912996
287.45682065912996 286.9247563158155
Average distance is : 178.03141387841106
Penalites are: 287.24300605626144
287.24300605626144 286.9247563158155
Average distance is : 178.08197785133692
Penalites are: 287.6398446974066
287.6398446974066 286.9247563158155
Average distance is : 178.03708711327033
Penalites are: 287.33473920859353
287.33473920859353 286.9247563158155
Average distance is : 178.01764914386425
Penalites are: 287.4568472064292
287.4568472064292 286.9247563158155
Average distance is : 178.0656993405132
Penalites are: 287.47692972402126
287.47692972402126 286.9247563158155


In [35]:

def hillclimber(num_iterations, joined, all_households, rel_poi_df, df_afstandn2, mod_max = 5, parameter='score', complicated=True, clean=True, use_count=False, save=True):
    """
    Function to perform repeated hillclimber. This can be added as a building block
    directly to the standard solution, but also after for example a random algorithm.
    The results are to be seen.
    """
    joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = analyze_candidate_solution(joined, all_households, rel_poi_df, df_afstandn2, clean=clean, use_count=use_count)

    if parameter == 'score':
        best = avg_distance + penalties
    if parameter == 'penalties':
        best = penalties
        
    hillclimber_dict = {}
    hillclimber_dict[0] = [avg_distance, penalties, best]

    r = copy.deepcopy(joined)
    for i in range(1, num_iterations+1):
        fractions = ['rest', 'plastic', 'papier', 'glas', 'textiel']
        no_modifications = random.randint(1, mod_max)
#         print(no_modifications)
        for j in range(no_modifications):
            valid = False
            while not valid:
                location_a = random.randint(0, r.shape[0]-1)
                fraction_a = random.choice(fractions)
                location_b = random.randint(0, r.shape[0]-1)
                fraction_b = random.choice(fractions)

                if r.at[location_a, fraction_b] > 0 and r.at[location_b, fraction_a] > 0\
                                                    and fraction_a != fraction_b:
                    r.at[location_a, fraction_a] = int(r.at[location_a, fraction_a]) + 1
                    r.at[location_a, fraction_b] = int(r.at[location_a, fraction_b]) - 1
                    r.at[location_b, fraction_a] = int(r.at[location_a, fraction_b]) - 1
                    r.at[location_b, fraction_b] = int(r.at[location_a, fraction_b]) + 1
                    valid = True

        joined_cluster_distance2, good_result_rich2, aansluitingen2, avg_distance2, penalties2 = analyze_candidate_solution(r, all_households, rel_poi_df, df_afstandn2, clean=clean, use_count=use_count)
        hillclimber_dict[i] = [avg_distance2, penalties2, best, no_modifications]
        if parameter == 'score':
            print(avg_distance2+penalties2, best)
            if avg_distance2+penalties2 < best:
                best = avg_distance2+penalties2
            else:
                r = copy.deepcopy(joined)
        if parameter == 'penalties':
            print(penalties2, best)
            if penalties2 < best:
                best = penalties2
            else:
                r = copy.deepcopy(joined)

    hill_df = pd.DataFrame.from_dict(hillclimber_dict, orient='index')
    hill_df = hill_df.rename(columns={0:'avg_distance', 1:'penalties', 2:'best', 3:'amount of modifications'})
    
    if save:
        today = str(pd.datetime.now().date()) + '-' + str(pd.datetime.now().hour)
        hill_df.to_csv('hillclimber' + today + '.csv')
        r.to_csv('hillclimber_best_config' + today + '.csv')

    return hill_df, r